In [ ]:
import warnings
from datetime import datetime

import dask_geopandas as dgpd
import geopandas as gpd
from intake import open_catalog
from ruamel.yaml import YAML

warnings.filterwarnings("ignore", message=".*initial implementation of Parquet.*")

In [ ]:
gpd.read_file(gpd.datasets.get_path("naturalearth_lowres")).to_parquet(
    "naturalearth_lowres.parquet"
)
gpd.read_parquet("naturalearth_lowres.parquet")

In [ ]:
d = {
    "metadata": {"version": 1},
    "sources": {
        "naturalearth_lowres": {
            "description": "Boundaries of countries from Natural Earth",
            "driver": "geoparquet",
            "args": {"urlpath": "naturalearth_lowres.parquet"},
        }
    },
}
yaml = YAML()
with open("cat.yaml", "w") as f:
    yaml.dump(d, f)
cat = open_catalog("cat.yaml")
gdf = cat.naturalearth_lowres.read()
gdf

In [ ]:
gdf["date"] = datetime(2022, 1, 1)
gdf.to_parquet("naturalearth_lowres_2022-01-01.parquet")
gdf["date"] = datetime(2022, 1, 2)
gdf.to_parquet("naturalearth_lowres_2022-01-02.parquet")

In [ ]:
!ls naturalearth_lowres_2022-01-*.parquet

In [ ]:
dgpd.read_parquet("naturalearth_lowres_2022-01-*.parquet")

In [ ]:
d = {
    "metadata": {"version": 1},
    "sources": {
        "naturalearth_lowres": {
            "description": "Boundaries of countries from Natural Earth",
            "driver": "geoparquet",
            "args": {"urlpath": "naturalearth_lowres.parquet"},
        },
        "naturalearth_lowres_appended_daily": {
            "description": "Appended daily data of countries from Natural Earth",
            "driver": "geoparquet",
            "args": {"urlpath": "naturalearth_lowres_2022-01-*.parquet"},
        },
    },
}
yaml = YAML()
with open("cat.yaml", "w") as f:
    yaml.dump(d, f)
cat = open_catalog("cat.yaml")
dgdf = cat.naturalearth_lowres_appended_daily.to_dask()
dgdf

In [ ]:
d = {
    "metadata": {"version": 1},
    "sources": {
        "naturalearth_lowres": {
            "description": "Boundaries of countries from Natural Earth",
            "driver": "geoparquet",
            "args": {"urlpath": "naturalearth_lowres.parquet"},
        },
        "naturalearth_lowres_daily": {
            "description": "Appended daily data of countries from Natural Earth",
            "driver": "geoparquet",
            "args": {"urlpath": "naturalearth_lowres_2022-01-*.parquet"},
        },
        "naturalearth_lowres_by_day": {
            "description": "Daily data of countries from Natural Earth",
            "driver": "pattern_cat",
            "args": {
                "driver": "geoparquet",
                "urlpath": "naturalearth_lowres_{date:%Y-%m-%d}.parquet",
                "listable": False,
            },
        },
    },
}
yaml = YAML()
with open("cat.yaml", "w") as f:
    yaml.dump(d, f)
cat = open_catalog("cat.yaml")
cat.naturalearth_lowres_by_day.get_entry(date=datetime(2022, 1, 1)).read()

In [ ]:
cat.naturalearth_lowres_by_day.get_entry(date=datetime(2022, 1, 2)).to_dask()